# Análisis de vías rurales

In [248]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [249]:
# Cargar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from EDA_cleaning_utils import tabla_frecuencia
from EDA_cleaning_utils import graficar_barras
from EDA_cleaning_utils import normalizar_columna
import hvplot.pandas
pd.set_option('display.max_columns', None)

In [250]:
df = pd.read_csv('collected_obra_data.csv')
df.head()

,CONTRACT_ID,ENTITY_NAME,DEPARTMENT,MUNICIPALITY_TYPE,PROCESS_TYPE,CONTRACT_OBJECT,OBJETC_DETAIL,ESTIMATED_COST_ORIG,CONTRACT_VALUE_ORIG,ADDITIONAL_COST_ORIG,FINAL_COST_ORIG,YEAR,CONTRACT_DATE,START_DATE,ORIGINAL_DEADLINE,END_DATE,ID_ADJUDICACION,URLPROCESO,NIT_ENTIDAD,ID_CONTRATISTA,ESTIMATED_COST_NORM,CONTRACT_VALUE_NORM,ADDITIONAL_COST_NORM,FINAL_COST_NORM,ADDITIONAL_TIME,FINAL_DEADLINE,PROJECT_INTENSITY_NORM,PROJECT_INTENSITY_ORIG,AWARD_GROWTH_NORM,AWARD_GROWTH_ORIG,COST_DEVIATION_ORIG,COST_DEVIATION_NORM,TIME_DEVIATION,OWNER,REGION,GROUP,COST_GROUP_QUANTILE
0,17-13-6626390-6041040,NARINO ALCALDIA MUNICIPIO DE ARBOLEDA BERRUECOS,NARIÑO,TYPE_2,CONTRATACIÓN MÍNIMA CUANTÍA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",CONTRATAR LA OBRA PUBLICA PARA REALIZAR EL MAN...,1.999626e+07,1.999626e+07,0.0,1.999626e+07,2017,2017-05-26,2017-05-26,7,2017-06-02,6041040,{'url': 'https://www.contratos.gov.co/consulta...,800093386-8,1018464415,27.105601,27.105601,0.000000,27.105601,0,7,3.872229,2.856609e+06,0.000000,0.000000,0.000000,0.000000,0.0,MUNICIPALITY,PACIFICA,Vias Rurales hasta 2020,2
1,16-12-5894032-5360440,CUNDINAMARCA INSTITUTO DE INFRAESTRUCTURA Y C...,BOGOTÁ DC,OTHER,CONTRATACIÓN DIRECTA (LEY 1150 DE 2007),"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",AUNAR ESFUERZOS TECNICOS; ADMINISTRATIVOS Y FI...,1.493172e+08,1.493172e+08,0.0,1.493172e+08,2016,2016-12-07,2016-12-07,360,2017-12-07,5360440,{'url': 'https://www.contratos.gov.co/consulta...,900258711,899999466,216.572854,216.572854,0.000000,216.572854,0,360,0.601591,4.147701e+05,0.000000,0.000000,0.000000,0.000000,0.0,DEPARTMENT_GOVERNMENT,OTRA,Vias Rurales hasta 2020,4
2,19-1-205607-10520942,ANTIOQUIA ALCALDIA MUNICIPIO DE EL PENOL,ANTIOQUIA,TYPE_6,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",FRESADA Y PAVIMENTACION DE LA VIA DE ACCESO PR...,1.104422e+09,1.084239e+09,250060500.0,1.334300e+09,2019,2019-11-28,2019-12-04,60,2020-05-04,10520942,{'url': 'https://www.contratos.gov.co/consulta...,890980917,901337744,1333.656541,1309.283977,301.963131,1611.247108,90,150,21.821400,1.807065e+07,-1.827499,-1.827499,0.230632,0.230632,1.5,MUNICIPALITY,ANDINA,Vias Rurales hasta 2020,4
3,19-13-8824649-8190165,ANTIOQUIA ALCALDIA MUNICIPIO DE NECOCLI,ANTIOQUIA,TYPE_6,CONTRATACIÓN MÍNIMA CUANTÍA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",MANTENIMIENTO VIA ACCESO AL COLISEO DEL MUNIC...,2.150000e+07,2.150000e+07,0.0,2.150000e+07,2019,2019-01-03,2019-01-22,10,2019-02-01,8190165,{'url': 'https://www.contratos.gov.co/consulta...,890985354-8,900821698,25.962546,25.962546,0.000000,25.962546,0,10,2.596255,2.150000e+06,0.000000,0.000000,0.000000,0.000000,0.0,MUNICIPALITY,ANDINA,Vias Rurales hasta 2020,2
4,19-12-9611888-8756837,CAUCA ALCALDIA MUNICIPIO DE PIAMONTE,CAUCA,TYPE_6,CONTRATACIÓN DIRECTA (LEY 1150 DE 2007),"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",APOYO AL CONVENIO SOLIDARIO PARA EL MEJORAMIEN...,2.000000e+07,2.000000e+07,0.0,2.000000e+07,2019,2019-06-26,2019-06-26,30,2019-07-26,8756837,{'url': 'https://www.contratos.gov.co/consulta...,891856464-0,901276910,24.151206,24.151206,0.000000,24.151206,0,30,0.805040,6.666667e+05,0.000000,0.000000,0.000000,0.000000,0.0,MUNICIPALITY,PACIFICA,Vias Rurales hasta 2020,2


## Filtros

In [251]:
df.groupby('GROUP').size()

GROUP
Vias Rurales 2021-2023         1998
Vias Rurales hasta 2020        7522
Vias secundarias 2021-2023      119
Vias secundarias hasta 2020    1981
dtype: int64

In [252]:
# Quitar vías secundarías
df = df.loc[df['GROUP'].isin(['Vias Rurales 2021-2023','Vias Rurales hasta 2020'])].copy()

In [253]:
df.shape

(9520, 37)

In [254]:
df['OBJETC_DETAIL'] = df['OBJETC_DETAIL'].str.upper()

In [255]:
# Quitar convenios interadministrativos
kw_convenios = [
    'AUNAR',
    'CONVENIO',
    'ANUAR',
    'AUNNAR',
    'INTERADMINISTRATIVO',
    'COMPLEMENTACION DE ESFUERZOS INSTITUCIONALES',
    'COMPLEMENTAR ESFUERZOS INSTITUCIONALES',
    'UNIR  ESFUERZOS',
    'UNIR ESFUERZOS',
    'UNION DE ESFUERZOS'
               ]
# Unir las palabras clave en un solo string separadas por '|'
kw_convenios = '|'.join(kw_convenios)
df = df.loc[~df['OBJETC_DETAIL'].str.contains(kw_convenios)].copy()
df.shape

(6058, 37)

In [256]:
# Quitar contratos que no son obra
kw_otros = ['ESTUDIOS',
             'DISENOS',
             'ESTUDIO',
             'DISENO',
             'INTERVENTORIA',
             'INTERVENTORA',
             'INTRERVENTORIA',
             'ADMINISTRACION',
             'INVENTARIO',
             'ELABORACION DE UN  MANUAL',
             'SUMINISTRO',
             'INSTITUCIONES EDUCATIVAS',
             'INSTITUCION EDUCATIVA',
             'MANTENIMIENTO RUTINARIO']

kw_otros = '|'.join(kw_otros)

df = df.loc[~df['OBJETC_DETAIL'].str.contains(kw_otros)].copy()
df.shape

(4929, 37)

In [257]:
# Quitar otros tipos de obras
kw_otras_obras = ['ACUEDUCTO',
                  'ALCANTARILLADO',
                  'PARQUE',
                  'COLEGIO']

kw_otras_obras = '|'.join(kw_otras_obras)

df = df.loc[~df['OBJETC_DETAIL'].str.contains(kw_otras_obras)].copy()
df.shape

(4791, 37)

In [258]:
df = df.loc[~(df['PROCESS_TYPE'] == 'CONCURSO DE MÉRITOS ABIERTO')].copy()
df.shape

(4790, 37)

In [259]:
# Considerar solo contratos de mas de 100 millones
df = df[df['CONTRACT_VALUE_ORIG'] >= 1e8].copy()
df.shape

(1877, 37)

## Procesamiento

In [260]:
# Estandarizar la modalidad de contratación
modalidad_map = {
    'LICITACIÓN PÚBLICA':'LICITACIÓN', 
    'CONTRATACIÓN DIRECTA (LEY 1150 DE 2007)':'CONTRATACION DIRECTA',
    'SELECCIÓN ABREVIADA DE MENOR CUANTÍA (LEY 1150 DE 2007)':'SELECCIÓN ABREVIADA',
    'LICITACIÓN OBRA PÚBLICA':'LICITACIÓN', 
    'SELECCIÓN ABREVIADA DEL LITERAL H DEL NUMERAL 2 DEL ARTÍCULO 2 DE LA LEY 1150 DE 2007':'SELECCIÓN ABREVIADA'
}
df['PROCESS_TYPE'].replace(modalidad_map, inplace=True)

In [261]:
df.groupby('PROCESS_TYPE').size().sort_values(ascending=False)

PROCESS_TYPE
CONTRATACION DIRECTA    824
LICITACIÓN              508
SELECCIÓN ABREVIADA     432
RÉGIMEN ESPECIAL        113
dtype: int64

In [262]:
df.isnull().sum()

CONTRACT_ID               0
ENTITY_NAME               0
DEPARTMENT                0
MUNICIPALITY_TYPE         0
PROCESS_TYPE              0
CONTRACT_OBJECT           0
OBJETC_DETAIL             0
ESTIMATED_COST_ORIG       0
CONTRACT_VALUE_ORIG       0
ADDITIONAL_COST_ORIG      0
FINAL_COST_ORIG           0
YEAR                      0
CONTRACT_DATE             0
START_DATE                0
ORIGINAL_DEADLINE         0
END_DATE                  0
ID_ADJUDICACION           0
URLPROCESO                0
NIT_ENTIDAD               0
ID_CONTRATISTA            0
ESTIMATED_COST_NORM       0
CONTRACT_VALUE_NORM       0
ADDITIONAL_COST_NORM      0
FINAL_COST_NORM           0
ADDITIONAL_TIME           0
FINAL_DEADLINE            0
PROJECT_INTENSITY_NORM    0
PROJECT_INTENSITY_ORIG    0
AWARD_GROWTH_NORM         0
AWARD_GROWTH_ORIG         0
COST_DEVIATION_ORIG       0
COST_DEVIATION_NORM       0
TIME_DEVIATION            0
OWNER                     0
REGION                    0
GROUP               

In [282]:
dep_mod = {'BOGOT DC':'BOGOTÁ DC',
            'BOYAC':'BOYACÁ',
            'CAQUET':'CAQUETÁ',
            'CRDOBA':'CÓRDOBA',
            'SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA':'SAN ANDRÉS PROV. Y STA. CAT.',
            'BOLVAR':'BOLÍVAR',
            'CHOC':'CHOCÓ'}
df['DEPARTMENT'].replace(dep_mod, inplace=True)

## Análisis exploratorio de datos

### Frecuencia de desviación en costo

In [263]:
# Crear una variable que indique el proyecto tiene desviación en costo
df['COST_DEVIATION'] = df['ADDITIONAL_COST_ORIG'] > 0

In [264]:
# Tabla de frecuencia de la desviación en costo
tabla_frecuencia(df, 'COST_DEVIATION')

,COST_DEVIATION,TOTAL CONTRATOS,PORCENTAJE
0,False,1565,83.38
1,True,312,16.62


In [268]:
(df['CONTRACT_VALUE_ORIG']/1e6).describe()

count      1877.000000
mean       1416.079606
std        6217.478521
min         100.000000
25%         108.249000
50%         200.000000
75%         550.000000
max      119039.104374
Name: CONTRACT_VALUE_ORIG, dtype: float64

In [273]:
# Crear la variable de tamaño del proyecto
df['CONTRACT_VALUE_RANGE_MILLONES'] = pd.cut(df['CONTRACT_VALUE_ORIG']/1e6, bins=[100,150,300,900,120000], include_lowest=True)

In [278]:
# Frecuencia de desviación por tamaño del proyecto
df.groupby(['CONTRACT_VALUE_RANGE_MILLONES'], observed=False)['COST_DEVIATION'].mean()*100

CONTRACT_VALUE_RANGE_MILLONES
(99.999, 150.0]       5.577689
(150.0, 300.0]       19.727891
(300.0, 900.0]       21.846154
(900.0, 120000.0]    31.284916
Name: COST_DEVIATION, dtype: float64

In [287]:
df.groupby(['REGION'], observed=True)['COST_DEVIATION'].mean().sort_values(ascending=False)

REGION
ANDINA       0.280753
ORINOQUIA    0.261682
AMAZONIA     0.259259
PACIFICA     0.165138
CARIBE       0.156863
OTRA         0.063901
Name: COST_DEVIATION, dtype: float64

In [289]:
df.groupby(['MUNICIPALITY_TYPE'], observed=True)['COST_DEVIATION'].mean()

MUNICIPALITY_TYPE
NACIONAL DESCENTRALIZADO    0.500000
OTHER                       0.081574
TYPE_1                      0.210526
TYPE_2                      0.315789
TYPE_3                      0.428571
TYPE_4                      0.280000
TYPE_5                      0.346667
TYPE_6                      0.252115
Name: COST_DEVIATION, dtype: float64

In [290]:
df.groupby(['PROCESS_TYPE'], observed=True)['COST_DEVIATION'].mean()

PROCESS_TYPE
CONTRATACION DIRECTA    0.030340
LICITACIÓN              0.285433
RÉGIMEN ESPECIAL        0.230088
SELECCIÓN ABREVIADA     0.268519
Name: COST_DEVIATION, dtype: float64

In [286]:
df.columns

Index(['CONTRACT_ID', 'ENTITY_NAME', 'DEPARTMENT', 'MUNICIPALITY_TYPE',
       'PROCESS_TYPE', 'CONTRACT_OBJECT', 'OBJETC_DETAIL',
       'ESTIMATED_COST_ORIG', 'CONTRACT_VALUE_ORIG', 'ADDITIONAL_COST_ORIG',
       'FINAL_COST_ORIG', 'YEAR', 'CONTRACT_DATE', 'START_DATE',
       'ORIGINAL_DEADLINE', 'END_DATE', 'ID_ADJUDICACION', 'URLPROCESO',
       'NIT_ENTIDAD', 'ID_CONTRATISTA', 'ESTIMATED_COST_NORM',
       'CONTRACT_VALUE_NORM', 'ADDITIONAL_COST_NORM', 'FINAL_COST_NORM',
       'ADDITIONAL_TIME', 'FINAL_DEADLINE', 'PROJECT_INTENSITY_NORM',
       'PROJECT_INTENSITY_ORIG', 'AWARD_GROWTH_NORM', 'AWARD_GROWTH_ORIG',
       'COST_DEVIATION_ORIG', 'COST_DEVIATION_NORM', 'TIME_DEVIATION', 'OWNER',
       'REGION', 'GROUP', 'COST_GROUP_QUANTILE', 'COST_DEVIATION',
       'CONTRACT_VALUE_RANGE_MILLONES'],
      dtype='object')

In [293]:
df[df['PROCESS_TYPE'] == 'LICITACIÓN']['OBJETC_DETAIL'].to_clipboard()

In [298]:
df['TIPO_OBRA_CONSTRUCCION'] = df['OBJETC_DETAIL'].str.contains('CONSTRUCCION|PAVIMENTACION|COSNTRUCCION|COSNTRUCCION|CONTRUCCION')
df['TIPO_OBRA_CONSTRUCCION'].sum() 

556

In [299]:
df['TIPO_OBRA_MANTENIMIENTO'] = df['OBJETC_DETAIL'].str.contains('MANTENIMIENTO')
df['TIPO_OBRA_MANTENIMIENTO'].sum() 

342

In [300]:
df['TIPO_OBRA_REHABILITACION'] = df['OBJETC_DETAIL'].str.contains('REHABILITACION')
df['TIPO_OBRA_REHABILITACION'].sum() 

139

In [315]:
df['TIPO_OBRA_MEJORAMIENTO'] = df['OBJETC_DETAIL'].str.contains('MEJORAMIENTO')
df['TIPO_OBRA_MEJORAMIENTO'].sum() 

1078

In [316]:
df['TOTAL_CLASS_TIPO'] = np.int32(df['TIPO_OBRA_CONSTRUCCION'] + df['TIPO_OBRA_MANTENIMIENTO'] + df['TIPO_OBRA_REHABILITACION'] + df['TIPO_OBRA_MEJORAMIENTO'])

In [317]:
df['TOTAL_CLASS_TIPO'].value_counts()

TOTAL_CLASS_TIPO
1    1715
0     162
Name: count, dtype: int64

In [319]:
df['TIPO_OBRA_OTROS'] = df['TOTAL_CLASS_TIPO'] == 0
df['TIPO_OBRA_OTROS'].sum() 

162

In [325]:
df.columns[pd.Series(df.columns).str.contains('TIPO_OBRA')]

Index(['TIPO_OBRA_CONSTRUCCION', 'TIPO_OBRA_MANTENIMIENTO',
       'TIPO_OBRA_REHABILITACION', 'TIPO_OBRA_MEJORAMIENTO',
       'TIPO_OBRA_OTROS'],
      dtype='object')

In [336]:
cols_tipo_obra = ['TIPO_OBRA_CONSTRUCCION', 'TIPO_OBRA_MANTENIMIENTO',
       'TIPO_OBRA_REHABILITACION', 'TIPO_OBRA_MEJORAMIENTO',
       'TIPO_OBRA_OTROS']
df['TYPE_WORK'] = ''
for i in cols_tipo_obra:
    df.loc[df[i] == True, 'TYPE_WORK'] = i

In [337]:
df.groupby(['TYPE_WORK'])['COST_DEVIATION'].mean()

TYPE_WORK
TIPO_OBRA_CONSTRUCCION      0.231362
TIPO_OBRA_MANTENIMIENTO     0.278912
TIPO_OBRA_MEJORAMIENTO      0.113173
TIPO_OBRA_OTROS             0.259259
TIPO_OBRA_REHABILITACION    0.168317
Name: COST_DEVIATION, dtype: float64

In [344]:
# Gráfica de intervalos
# Asumiendo que df es tu DataFrame y cat_cols son tus variables categóricas de interés
cat_cols = ['CONTRACT_VALUE_RANGE_MILLONES','REGION','MUNICIPALITY_TYPE','PROCESS_TYPE','TYPE_WORK']
y_var = 'COST_DEVIATION'

resumen = pd.DataFrame()

# Función para calcular la desviación estándar para una variable binaria
def std_binaria(p, n):
    return np.sqrt(p * (1 - p) / n)

# Función para calcular el intervalo de confianza del 95% para una proporción
def confianza_intervalo(p, n):
    z = 1.96  # Valor z para un intervalo de confianza del 95%
    margin = z * std_binaria(p, n)
    return margin

# Inicializar la lista para guardar los resultados de la prueba Chi-cuadrado
chi2_resultados = []

for cat in cat_cols:
    # Agrupar por cat, calcular tamaño y proporción
    temp = (
        df
        .groupby([cat])[y_var]
        .agg(['size','sum','mean'])
        .rename(columns={'size': 'Total Contratos', 'sum':'Total Contratos con Desviación','mean': 'Proporción de Desviación en Costo'})
    )
    
    # Calcular desviación estándar para la variable binaria y los intervalos de confianza
    temp['Desv. Estandar'] = temp.apply(lambda row: std_binaria(row['Proporción de Desviación en Costo'], row['Total Contratos']), axis=1)
    temp['CI 95% Inferior'] = temp['Proporción de Desviación en Costo'] - temp.apply(lambda row: confianza_intervalo(row['Proporción de Desviación en Costo'], row['Total Contratos']), axis=1)
    temp['CI 95% Superior'] = temp['Proporción de Desviación en Costo'] + temp.apply(lambda row: confianza_intervalo(row['Proporción de Desviación en Costo'], row['Total Contratos']), axis=1)
    
    # Ajustar valores IC Inferior < 0 e IC Superior > 1
    temp['CI 95% Inferior'] = temp['CI 95% Inferior'].apply(lambda x: np.max([0,x]))
    temp['CI 95% Superior'] = temp['CI 95% Superior'].apply(lambda x: np.min([1,x]))

    
    # Ordenar solo variables categóricas nominales
    if cat in ['REGION','PROCESS_TYPE','TYPE_WORK']:
       temp =  temp.sort_values(by='Total Contratos', ascending=False)
    
    # Preparar los datos para la prueba Chi-cuadrado
    crosstab = pd.crosstab(df[cat], df[y_var])
    
    # Realizar la prueba Chi-cuadrado y obtener el p-valor
    chi2, p, dof, ex = stats.chi2_contingency(crosstab)
    #p = np.round(p,6)
    chi2_resultados.append((cat, chi2, p))

    
    # Reiniciar el índice para convertir el índice en una columna y facilitar la concatenación
    #temp.reset_index(inplace=True)
    temp['Variable Categorica'] = cat
    #temp['Chi2'] = chi2
    temp['p-value (chi2)'] = p
    
    # Concatenar el DataFrame temporal con el resumen
    resumen = pd.concat([resumen, temp])

# Ajustar el DataFrame resumen
resumen['Categorias'] = resumen.index
resumen = resumen.set_index(['Variable Categorica', 'p-value (chi2)','Categorias'])
resumen

/var/folders/yv/nqf1_stj1lsb93l9v9j6z2h00000gn/T/ipykernel_15925/3741308483.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([cat])[y_var]


Total Contratos  \
Variable Categorica           p-value (chi2) Categorias                                  
CONTRACT_VALUE_RANGE_MILLONES 2.867297e-28   (99.999, 150.0]                       753   
                                             (150.0, 300.0]                        441   
                                             (300.0, 900.0]                        325   
                                             (900.0, 120000.0]                     358   
REGION                        8.426363e-29   OTRA                                  892   
                                             ANDINA                                691   
                                             PACIFICA                              109   
                                             ORINOQUIA                             107   
                                             CARIBE                                 51   
                                             AMAZONIA                               27   
MUNICIPALITY_TYPE             1.108124e-25   NACIONAL DESCENTRALIZADO                2   
                                             OTHER                                1042   
                                             TYPE_1                                 38   
                                             TYPE_2                                 76   
                                             TYPE_3                                 28   
                                             TYPE_4                                 25   
                                             TYPE_5                                 75   
                                             TYPE_6                                591   
PROCESS_TYPE                  1.253091e-42   CONTRATACION DIRECTA                  824   
                                             LICITACIÓN                            508   
                                             SELECCIÓN ABREVIADA                   432   
                                             RÉGIMEN ESPECIAL                      113   
TYPE_WORK                     1.024699e-11   TIPO_OBRA_MEJORAMIENTO               1078   
                                             TIPO_OBRA_CONSTRUCCION                389   
                                             TIPO_OBRA_OTROS                       162   
                                             TIPO_OBRA_MANTENIMIENTO               147   
                                             TIPO_OBRA_REHABILITACION              101   

                                                                       Total Contratos con Desviación  \
Variable Categorica           p-value (chi2) Categorias                                                 
CONTRACT_VALUE_RANGE_MILLONES 2.867297e-28   (99.999, 150.0]                                       42   
                                             (150.0, 300.0]                                        87   
                                             (300.0, 900.0]                                        71   
                                             (900.0, 120000.0]                                    112   
REGION                        8.426363e-29   OTRA                                                  57   
                                             ANDINA                                               194   
                                             PACIFICA                                              18   
                                             ORINOQUIA                                             28   
                                             CARIBE                                                 8   
                                             AMAZONIA                                               7   
MUNICIPALITY_TYPE             1.108124e-25   NACIONAL DESCENTRALIZADO                               1   
                                             OTHER                                             